In [1]:
cd ../..

/home/nazif/thesis/mirscribe-vcf


/home/nazif/.cache/pypoetry/virtualenvs/mirscribe-vcf-8VJm4tPg-py3.10/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import json
import pandas as pd
from sqlalchemy import create_engine
pyensembl = create_engine('sqlite:////home/nazif/.cache/pyensembl/GRCh38/ensembl111/Homo_sapiens.GRCh38.111.gtf.db')


In [3]:

with open("data/gsea/h.all.v2023.2.Hs.json") as file:
    gsea_data = json.load(file)

# Create a dictionary to store the data
data_dict = {}

# Iterate over each hallmark property
for hallmark, hallmark_data in gsea_data.items():
    # Extract the gene symbols for the current hallmark
    gene_symbols = hallmark_data['geneSymbols']
    
    # Add the gene symbols as a column in the data dictionary
    data_dict[hallmark] = pd.Series(True, index=gene_symbols)

# Create the DataFrame from the data dictionary
gsea = pd.DataFrame(data_dict)

# Fill missing values with False
gsea.fillna(False, inplace=True)

# Reset the index to make gene IDs as rows
gsea.reset_index(inplace=True)
gsea.rename(columns={'index': 'gene_name'}, inplace=True)

def clean_column_name(col_name):
    if col_name.startswith("HALLMARK_"):
        return "is_" + col_name[len("HALLMARK_"):].lower()
    return col_name.lower()

# Apply the function to all column names
gsea.rename(columns=lambda col: clean_column_name(col), inplace=True)

gsea

/tmp/ipykernel_33248/2886532373.py:19: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  gsea.fillna(False, inplace=True)


,gene_name,is_tnfa_signaling_via_nfkb,is_hypoxia,is_cholesterol_homeostasis,is_mitotic_spindle,is_wnt_beta_catenin_signaling,is_tgf_beta_signaling,is_il6_jak_stat3_signaling,is_dna_repair,is_g2m_checkpoint,...,is_heme_metabolism,is_coagulation,is_il2_stat5_signaling,is_bile_acid_metabolism,is_peroxisome,is_allograft_rejection,is_spermatogenesis,is_kras_signaling_up,is_kras_signaling_dn,is_pancreas_beta_cells
0,A2M,False,False,False,False,False,False,True,False,False,...,False,True,False,False,False,False,False,False,False,False
1,AAAS,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
2,AADAT,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,AARS1,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
4,ABAT,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4379,ZNRF4,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
4380,ZPBP,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
4381,ZW10,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4382,ZWINT,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False


In [4]:
genes = pd.read_sql("gene", pyensembl, columns=["gene_id", "gene_name"])
df = gsea.merge(genes, on="gene_name", how="left")


In [5]:
df[df.gene_id.isna()]


,gene_name,is_tnfa_signaling_via_nfkb,is_hypoxia,is_cholesterol_homeostasis,is_mitotic_spindle,is_wnt_beta_catenin_signaling,is_tgf_beta_signaling,is_il6_jak_stat3_signaling,is_dna_repair,is_g2m_checkpoint,...,is_coagulation,is_il2_stat5_signaling,is_bile_acid_metabolism,is_peroxisome,is_allograft_rejection,is_spermatogenesis,is_kras_signaling_up,is_kras_signaling_dn,is_pancreas_beta_cells,gene_id
2386,METTL7B,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,NaN
4008,THEG,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,NaN


In [6]:
# manually taken from ENSEMBL
df.loc[df.gene_name == 'METTL7B', 'gene_id'] = 'ENSG00000163575'
df.loc[df.gene_name == 'THEG', 'gene_id'] = 'ENSG00000105549'

In [7]:
df.isna().sum()

gene_name                               0
is_tnfa_signaling_via_nfkb              0
is_hypoxia                              0
is_cholesterol_homeostasis              0
is_mitotic_spindle                      0
is_wnt_beta_catenin_signaling           0
is_tgf_beta_signaling                   0
is_il6_jak_stat3_signaling              0
is_dna_repair                           0
is_g2m_checkpoint                       0
is_apoptosis                            0
is_notch_signaling                      0
is_adipogenesis                         0
is_estrogen_response_early              0
is_estrogen_response_late               0
is_androgen_response                    0
is_myogenesis                           0
is_protein_secretion                    0
is_interferon_alpha_response            0
is_interferon_gamma_response            0
is_apical_junction                      0
is_apical_surface                       0
is_hedgehog_signaling                   0
is_complement                     

In [9]:
df.to_csv("data/gsea/gsea.csv", index=False)